In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
train_file = "./train.csv"
test_file = "./test.csv"
num_cols = ["ps_reg_01", "ps_reg_02", "ps_reg_03","ps_car_12", "ps_car_13", "ps_car_14", "ps_car_15"]
ignore_cols = ["id", "target", "ps_calc_01", "ps_calc_02", "ps_calc_03", "ps_calc_04", "ps_calc_05", 
               "ps_calc_06", "ps_calc_07", "ps_calc_08", "ps_calc_09", "ps_calc_10", "ps_calc_11", 
               "ps_calc_12", "ps_calc_13", "ps_calc_14","ps_calc_15_bin", "ps_calc_16_bin", 
               "ps_calc_17_bin","ps_calc_18_bin", "ps_calc_19_bin", "ps_calc_20_bin"]

In [3]:
cfg = {
    "feature_size": None,
    "field_size": None,
    "embed_size":8,
    "deep_nn":[32,32],
    "dropout_fm": 0,
    "dropout_deep": 0.2,
    "epoch":20,
    "batch":10000,
    "split": 0.1,
    "class_weight": None 
}

In [4]:
def overview(cfg):    
    dfTrain = pd.read_csv(train_file)
    dfTest = pd.read_csv(test_file)
    df = pd.concat([dfTrain,dfTest], sort=False)

    field_size = len(df.columns) - len(ignore_cols)
    feature_dict = {}
    feature_size = 0
    for col in df.columns:
        if col in ignore_cols:
            continue
        elif col in num_cols:
            feature_dict[col] = feature_size
            feature_size += 1
        else:
            unique_val = df[col].unique()
            feature_dict[col] = dict(zip(unique_val,range(feature_size,len(unique_val) + feature_size)))
            feature_size += len(unique_val)
    
    cfg['field_size'] = field_size
    cfg['feature_size'] = feature_size
    return dfTrain, feature_dict

In [5]:
dfTrain, feature_dict = overview(cfg)

In [6]:
def preprocess(train_df, cfg):
    label_df = train_df[['target']]
    neg, pos = np.bincount(label_df.values.flatten())
    cfg["class_weight"] = {0: (1/neg)*(neg+pos), 1: (1/pos)*(neg+pos)}
    
    train_df.drop(['target','id'],axis=1,inplace=True)
    feature_idx = train_df.copy()
    feature_val = train_df.copy()
    for col in feature_idx.columns:
        if col in ignore_cols:
            feature_idx.drop(col,axis=1,inplace=True)
            feature_val.drop(col,axis=1,inplace=True)
            continue
        elif col in num_cols:
            feature_idx[col] = feature_dict[col]
        else:
            feature_idx[col] = feature_idx[col].map(feature_dict[col])
            feature_val[col] = 1        
            
    split_idx = feature_idx.shape[0] - round(feature_idx.shape[0]*cfg["split"])
    train_input = [feature_idx[:split_idx].values, feature_val[:split_idx].values]
    train_y = label_df[:split_idx].values
    validate_input = [feature_idx[split_idx:].values, feature_val[split_idx:].values]
    validate_y = label_df[split_idx:].values

    return train_input, train_y, validate_input, validate_y

In [7]:
train_input, train_y, validate_input, validate_y = preprocess(dfTrain, cfg)

In [8]:
class DeepFM(tf.keras.Model):
    def __init__(self, cfg):
        super(DeepFM, self).__init__()
        self.feature_size = cfg['feature_size']
        self.field_size = cfg['field_size']
        self.embed_size = cfg['embed_size']
        self.deep_nn = cfg['deep_nn']
        
        self.dropout_fm = cfg['dropout_fm']
        self.dropout_deep = cfg['dropout_deep']
        
        # fm        
        self.feature_weight = tf.keras.layers.Embedding(cfg['feature_size'], 1)
        self.feature_embed = tf.keras.layers.Embedding(cfg['feature_size'], cfg['embed_size'])

        # dnn
        for layer in range(len(cfg['deep_nn'])):
            setattr(self, 'dense_' + str(layer), tf.keras.layers.Dense(self.deep_nn[layer]))
            setattr(self, 'batchNorm_' + str(layer), tf.keras.layers.BatchNormalization())
            setattr(self, 'activation_' + str(layer), tf.keras.layers.Activation('relu'))
            setattr(self, 'dropout_' + str(layer), tf.keras.layers.Dropout(self.dropout_deep))
            
        self.fc = tf.keras.layers.Dense(1, activation='sigmoid', use_bias=True)

    def call(self, inputs, training=True):
        # inputs = [feature_idx, feature_val]
        reshaped_feature_val = tf.cast(tf.reshape(inputs[1], shape=[-1,self.field_size,1]), tf.float32)
        # linear        
        weights = self.feature_weight(inputs[0])
        linear = tf.reduce_sum(tf.multiply(weights,reshaped_feature_val),2)
        
        # fm  
        embeddings = self.feature_embed(inputs[0])
        second_inner = tf.multiply(embeddings,reshaped_feature_val)
        
        summed_features_emb = tf.reduce_sum(second_inner,1)
        summed_features_emb_square = tf.square(summed_features_emb)
        
        squared_features_emb = tf.square(second_inner)
        squared_sum_features_emb = tf.reduce_sum(squared_features_emb,1)
        
        fm = 0.5 * tf.subtract(summed_features_emb_square,squared_sum_features_emb)
        
        # dnn
        y_deep = tf.reshape(embeddings,shape=[-1,self.field_size * self.embed_size])
        for layer in range(0, len(self.deep_nn)):
            y_deep = getattr(self, 'dense_' + str(layer))(y_deep)
            y_deep = getattr(self, 'batchNorm_' + str(layer))(y_deep, training=training)
            y_deep = getattr(self, 'activation_' + str(layer))(y_deep)
            y_deep = getattr(self, 'dropout_' + str(layer))(y_deep, training=training)
            
        # concat
        concat = tf.concat([linear, fm, y_deep], axis=1)                                
        out = self.fc(concat)
        return out

In [9]:
model = DeepFM(cfg)
# model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.compile(optimizer = 'adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['binary_accuracy', 'AUC'])
model.fit(train_input, train_y, epochs=cfg['epoch'], batch_size=cfg['batch'], shuffle=True, class_weight=cfg['class_weight'],
          verbose=1, validation_data=(validate_input, validate_y))

Train on 535691 samples, validate on 59521 samples
Epoch 1/20
535691/535691 [==============================] - 6s 11us/sample - loss: 1.4137 - binary_accuracy: 0.6746 - AUC: 0.5797 - val_loss: 1.4057 - val_binary_accuracy: 0.9633 - val_AUC: 0.5076
Epoch 2/20
535691/535691 [==============================] - 1s 2us/sample - loss: 1.3665 - binary_accuracy: 0.8952 - AUC: 0.6099 - val_loss: 1.3911 - val_binary_accuracy: 0.9633 - val_AUC: 0.5355
Epoch 3/20
535691/535691 [==============================] - 1s 2us/sample - loss: 1.3604 - binary_accuracy: 0.8923 - AUC: 0.6227 - val_loss: 1.3883 - val_binary_accuracy: 0.9633 - val_AUC: 0.5812
Epoch 4/20
535691/535691 [==============================] - 1s 2us/sample - loss: 1.3585 - binary_accuracy: 0.8829 - AUC: 0.6269 - val_loss: 1.3858 - val_binary_accuracy: 0.9633 - val_AUC: 0.6023
Epoch 5/20
535691/535691 [==============================] - 1s 2us/sample - loss: 1.3568 - binary_accuracy: 0.8754 - AUC: 0.6305 - val_loss: 1.3830 - val_binary_acc

In [10]:
def prepredict(path):
    predict_df = pd.read_csv(path)
    feature_idx = predict_df.copy()
    feature_val = predict_df.copy()
    for col in feature_idx.columns:
        if col in ignore_cols:
            feature_idx.drop(col,axis=1,inplace=True)
            feature_val.drop(col,axis=1,inplace=True)
            continue
        elif col in num_cols:
            feature_idx[col] = feature_dict[col]
        else:
            feature_idx[col] = feature_idx[col].map(feature_dict[col])
            feature_val[col] = 1 
    return [feature_idx.values, feature_val.values]

In [11]:
# predict_input = prepredict(test_file)
# label_pre = model.predict(predict_input)

In [12]:
validate_input

[array([[  3,   8,  20, ..., 254, 255, 256],
        [  0,   8,  19, ..., 254, 255, 256],
        [  1,   9,  22, ..., 254, 255, 256],
        ...,
        [  1,   9,  24, ..., 254, 255, 256],
        [  2,   8,  19, ..., 254, 255, 256],
        [  3,   9,  23, ..., 254, 255, 256]]),
 array([[1.        , 1.        , 1.        , ..., 0.77539646, 0.39076847,
         3.31662479],
        [1.        , 1.        , 1.        , ..., 0.99036682, 0.32863353,
         3.60555128],
        [1.        , 1.        , 1.        , ..., 0.43593729, 0.35566838,
         0.        ],
        ...,
        [1.        , 1.        , 1.        , ..., 0.59637334, 0.39874804,
         1.73205081],
        [1.        , 1.        , 1.        , ..., 0.76443411, 0.38496753,
         3.16227766],
        [1.        , 1.        , 1.        , ..., 0.9326493 , 0.37802116,
         3.74165739]])]

In [13]:
validate_y[:40]

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [14]:
predict_for_vali = model.predict(validate_input)

In [15]:
predict_for_vali = pd.DataFrame(predict_for_vali)
label_for_vali = pd.DataFrame(validate_y)

In [16]:
predict_for_vali.shape

(59521, 1)

In [17]:
label_for_vali.shape

(59521, 1)

In [18]:
check_pd = pd.concat([predict_for_vali, label_for_vali], axis=1)

In [19]:
check_pd.columns = ['predict', 'true']

In [20]:
check_pd.head(40)

,predict,true
0,0.005041,0
1,0.831485,0
2,0.031188,0
3,0.102174,0
4,0.004742,0
5,0.010632,0
6,0.038799,0
7,0.000802,0
8,0.783564,0
9,0.003064,0


In [21]:
check_pd[(check_pd['true'] == 1) & (check_pd['predict'] >= 0.05)]

,predict,true
34,0.144259,1
114,0.958237,1
215,0.092884,1
255,0.990285,1
296,0.382946,1
...,...,...
59291,0.921292,1
59315,0.986605,1
59355,0.149215,1
59408,0.853664,1


In [22]:
check_pd[check_pd['true'] == 1]

,predict,true
34,0.144259,1
114,0.958237,1
120,0.009200,1
121,0.046208,1
131,0.002316,1
...,...,...
59315,0.986605,1
59355,0.149215,1
59408,0.853664,1
59422,0.096931,1


In [23]:
sort = check_pd.sort_values("predict",ascending=False)

In [24]:
sort[:40]

,predict,true
49062,0.999984,0
56473,0.999969,0
35372,0.999959,0
18553,0.999957,0
58241,0.999955,0
48156,0.999952,0
3113,0.999943,1
31050,0.999929,0
7853,0.999915,0
26736,0.999905,1


In [25]:
sort_true = check_pd.sort_values("true",ascending=False)
sort_true[:40]

,predict,true
29760,0.002943,1
6630,0.016401,1
6573,0.209917,1
36206,0.307185,1
36212,0.000062,1
53175,0.004709,1
36214,0.102430,1
6594,0.990063,1
36216,0.988915,1
6600,0.010699,1


In [26]:
sort_false = check_pd.sort_values("true",ascending=True)
sort_false[:40]

,predict,true
0,0.005041,0
39305,0.000492,0
39306,0.000558,0
39307,0.004914,0
39308,0.000229,0
39309,0.017649,0
39310,0.002192,0
39311,0.000928,0
39312,0.036864,0
39313,0.032936,0
